In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import math
import tqdm
import numpy as np
import pandas as pd

torch.manual_seed(192837)
rand = random.Random(192838)

In [2]:
CITY_COLUMN = 'city_ascii'
COUNTRY_COLUMN = 'iso2'

In [3]:
df_all = pd.read_csv('../emb/data/worldcities.csv')

'''
df_top_countries = df_all.groupby(COUNTRY_COLUMN).size().sort_values(ascending=False).reset_index()
classes = list(df_top_countries.head(N_CLASSES)[COUNTRY_COLUMN])
'''
classes = ['IN', 'US', 'BR', 'DE', 'CN', 'JP', 'RU', 'MX']
N_CLASSES = len(classes)

print(classes)
class_to_id = {classes[i]: i for i in range(N_CLASSES)}

df_all = df_all[df_all[COUNTRY_COLUMN].isin(classes)]
df_all = df_all[df_all[CITY_COLUMN].notna()]
df_all.head(10)

['IN', 'US', 'BR', 'DE', 'CN', 'JP', 'RU', 'MX']


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6870,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
6,Shanghai,Shanghai,31.2286,121.4747,China,CN,CHN,Shanghai,admin,24073000.0,1156073548
7,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,23086000.0,1076532519
9,Mexico City,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,21804000.0,1484247881
10,Kolkāta,Kolkata,22.5675,88.3700,India,IN,IND,West Bengal,admin,21747000.0,1356060520
14,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18832416.0,1840034016
15,Beijing,Beijing,39.9067,116.3975,China,CN,CHN,Beijing,primary,18522000.0,1156228865


In [4]:
character_set = set()
for city in df_all[CITY_COLUMN]:
    for ch in city:
        character_set.add(ch)

PAD_TOKEN = 0
tokens = ['<PAD>'] + sorted(list(character_set))
token_to_id = {tokens[i]: i for i in range(len(tokens))}
N_VOCAB = len(tokens)

''.join(tokens)

"<PAD> '()-./2ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [5]:
def encode(city: str, max_length: int):
    encoded = [token_to_id[city[i]] for i in range(min(max_length, len(city)))]
    return encoded + [PAD_TOKEN] * (max_length - len(encoded))

def decode(city_enc: list[int]):
    return ''.join([tokens[tk] for tk in city_enc if tk != PAD_TOKEN])

In [6]:
city_length_90_perc = np.quantile(np.array(list(map(len, df_all[CITY_COLUMN]))), 0.8)

SEQ_LENGTH = int(city_length_90_perc)
SEQ_LENGTH

12

In [7]:
data_list_x = [encode(city, SEQ_LENGTH) for city in df_all[CITY_COLUMN]]
data_list_y = [class_to_id[iso2] for iso2 in df_all[COUNTRY_COLUMN]]
data_list = list(zip(data_list_x, data_list_y))
rand.shuffle(data_list)

data_x = torch.tensor(np.array([x for x, _ in data_list], dtype=np.long))
data_y = torch.tensor(np.array([y for _, y in data_list], dtype=np.long))

for i in range(5):
    print(decode(data_x[i]), classes[data_y[i]])

TRAIN_TEST_SPLIT = 0.9
n_train = round(data_x.shape[0] * TRAIN_TEST_SPLIT)
train_x, train_y = data_x[:n_train, :], data_y[:n_train]
test_x,  test_y  = data_x[n_train:, :], data_y[n_train:]

print(data_x.shape[0], n_train, data_x.shape[0] - n_train)

Gohadi IN
Kandra IN
Chuanliaocun CN
Aracoiaba BR
Miryal IN
22320 20088 2232


In [8]:
class RNNClassifier(nn.Module):
    def __init__(self,
                 emb_dim: int,
                 hidden_dim: int):
        super().__init__()

        self.emb = nn.Linear(N_VOCAB, emb_dim, bias=False)

        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers=1, batch_first=True)
        self.head = nn.Linear(hidden_dim, N_CLASSES)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor):
        if len(x.shape) == 2:  # unbatched input
            x = x.unsqueeze(0)
        B, *_ = x.shape

        x = self.emb(x)
        rnn_o, h_n = self.rnn(x)
        o = self.head(h_n[-1])
        return o

In [9]:
def train_epoch(model: nn.Module,
                optimizer: optim.Optimizer,
                dataset_x: torch.Tensor,
                dataset_y: torch.Tensor,
                batch_size: int):
    model.train()
    
    loss_sum = 0
    accu_sum = 0
    n_batches = math.ceil(dataset_x.shape[0] / batch_size)

    for i in tqdm.tqdm(range(n_batches), 'train'):
        x, y = dataset_x[i * batch_size: (i+1) * batch_size, :], dataset_y[i * batch_size: (i+1) * batch_size]
        x = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
        y_hat = model.forward(x)

        loss = F.cross_entropy(y_hat, y)
        loss_sum += loss.detach().clone()

        with torch.no_grad():
            accu_sum += torch.sum(y_hat.argmax(dim=-1) == y) / x.shape[0]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    return loss_sum / n_batches, accu_sum / n_batches

def test_epoch(model: nn.Module, dataset_x: torch.Tensor, dataset_y: torch.Tensor, batch_size):
    model.eval()
    
    with torch.no_grad():
        loss_sum = 0
        accu_sum = 0
        n_batches = math.ceil(dataset_x.shape[0] / batch_size)

        for i in tqdm.tqdm(range(n_batches), ' test'):
            x, y = dataset_x[i * batch_size: (i+1) * batch_size, :], dataset_y[i * batch_size: (i+1) * batch_size]
            x = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
            y_hat = model.forward(x)

            loss = F.cross_entropy(y_hat, y)
            loss_sum += loss
            accu_sum += torch.sum(y_hat.argmax(dim=-1) == y) / x.shape[0]
        
        return loss_sum / n_batches, accu_sum / n_batches

In [10]:
model = RNNClassifier(emb_dim=16,
                      hidden_dim=32)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
N_EPOCHS = 50
BATCH_SIZE = 32

In [11]:
for epoch_i in range(N_EPOCHS):
    print(f'=== epoch {epoch_i} ===')

    train_loss, train_accu = train_epoch(model, optimizer, train_x, train_y, BATCH_SIZE)
    test_loss, test_accu = test_epoch(model, test_x, test_y, BATCH_SIZE)

    print('train loss:', train_loss.item())
    print(' test loss:', test_loss.item())
    print('train accu:', train_accu.item())
    print(' test accu:', test_accu.item())

=== epoch 0 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 336.29it/s]


train loss: 1.5864789485931396
 test loss: 1.3765908479690552
train accu: 0.4347963333129883
 test accu: 0.5248511433601379
=== epoch 1 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 244.46it/s]


train loss: 1.2607035636901855
 test loss: 1.2409735918045044
train accu: 0.5563130378723145
 test accu: 0.5565475821495056
=== epoch 2 ===


...

train loss: 0.445340096950531
 test loss: 0.5990995168685913
train accu: 0.8516786694526672
 test accu: 0.7934523820877075
=== epoch 48 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 593.14it/s]


train loss: 0.44174909591674805
 test loss: 0.5979602336883545
train accu: 0.8536193370819092
 test accu: 0.7965773940086365
=== epoch 49 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 272.74it/s]

train loss: 0.4382658004760742
 test loss: 0.5968939661979675
train accu: 0.855659544467926
 test accu: 0.7970238327980042


In [12]:
for i in range(20):
    print(decode(test_x[i]), classes[test_y[i]], end=' ')
    y_hat = torch.softmax(model.forward(F.one_hot(test_x[i], N_VOCAB).to(dtype=torch.float32)), dim=-1).squeeze()
    print(classes[y_hat.argmax(dim=-1)], f'({y_hat.max().item() * 100:.2f}%)')

Leme BR US (46.62%)
Minatitlan MX MX (59.94%)
Valley US US (86.04%)
Garden City US US (99.88%)
Hampton US US (95.88%)
Dicholi IN IN (91.98%)
Pau Brasil BR IN (76.90%)
Taloda IN IN (79.22%)
Mutum BR BR (38.58%)
Degana IN IN (58.15%)
Sao Goncalo  BR BR (87.28%)
Pileru IN IN (52.74%)
Hunsur IN IN (87.33%)
Vincennes US US (62.03%)
Lawrence US US (99.57%)
Bad Iburg DE DE (81.99%)
Morbi IN IN (61.50%)
Othello US US (85.89%)
Amari IN IN (53.31%)
Lobau DE US (40.05%)


In [15]:
from torch.distributions import Categorical
from torch.distributions.gumbel import Gumbel
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

class AntiEmb():
    def __init__(self,
               inner_net: nn.Module,
               desired_c: int,
               lr: float=1e-2,
               right_c_coef: float=1.0,
               wrong_c_coef: float=1.0):
        super().__init__()

        self.input_shape = (1, SEQ_LENGTH, N_VOCAB)
        self.input = nn.Parameter(
            data=torch.normal(0, 1, size=self.input_shape),
            requires_grad=True
        )

        self.inner_net = inner_net
        self.grad_eraser = torch.optim.SGD(self.inner_net.parameters())
        self.anti_optimizer = torch.optim.Adam((self.input,), lr=lr)
        # self.anti_scheduler = CosineAnnealingWarmRestarts(self.anti_optimizer, T_0=50, T_mult=2)

        self.desired_c = desired_c
        self.right_c_coef = right_c_coef
        self.wrong_c_coef = wrong_c_coef

        # self.input_min = 0.0
        # self.input_max = 1.0

        self.gumbel = Gumbel(loc=0.0, scale=1.0)
        self.epsilon = 1e-8

    def test(self):
        with torch.no_grad():
            return self.inner_net.forward(self.input)

    def zero_grad(self):
        self.grad_eraser.zero_grad()
        self.anti_optimizer.zero_grad()

    def crop(self):
        self.input.data.copy_(torch.clamp(self.input.detach().clone(), self.input_min, self.input_max))
    
    def step(self):
        prev_input = self.input.detach().clone()

        probs = torch.softmax(self.input, dim=-1)
        gumb = self.gumbel.sample(self.input_shape)

        j_onehot = F.one_hot(torch.argmax(gumb + torch.log(probs + self.epsilon), dim=-1), N_VOCAB)
        j_onehot = j_onehot.to(torch.float32).requires_grad_(True)

        j_continuous = torch.softmax(gumb + torch.log(probs + self.epsilon), dim=-1)

        output = self.inner_net.forward(j_onehot).squeeze()
        output_soft = torch.softmax(output, dim=-1)
        
        loss = output_soft[torch.arange(0, N_CLASSES) != self.desired_c].sum() * self.wrong_c_coef + \
               output_soft[self.desired_c] * -self.right_c_coef
        loss.backward()

        j_continuous.backward(j_onehot.grad)

        self.anti_optimizer.step()
        self.zero_grad()

        # self.anti_scheduler.step()

        # self.crop()

        return prev_input, output.detach().clone(), loss.detach().clone()

In [16]:
ANTI_NET_STEPS = 3000
TEST_TRIES = 5

for i in range(N_CLASSES):
    desired_c = i

    print(f'-=-=- step {i} -=-=-')
    print(f'des class:', classes[desired_c], f'({desired_c})')

    fin_loss = float('+inf')
    fin_inp, fin_outp = None, None

    for try_i in range(TEST_TRIES):
        anti_net = AntiEmb(model, desired_c, lr=5e-2)

        for step_i in tqdm.tqdm(range(ANTI_NET_STEPS), f'attempt #{try_i}'):
            inp, outp, loss = anti_net.step()
        
        if loss.item() < fin_loss:
            fin_loss = loss.item()
            fin_inp = inp
            fin_outp = outp
        
        print(f'attempt #{try_i} loss:', loss.item())
    
    print(f'fin loss:', fin_loss)
    print(f'fin output (from function):', fin_outp)

    inp_argmax = fin_inp.argmax(dim=-1)

    print(f'fin input  (argmax):', decode(inp_argmax.squeeze()))
    print(f'fin output (argmax):', model.forward(F.one_hot(inp_argmax, N_VOCAB).to(torch.float32)))

    dist = torch.distributions.Categorical(logits=fin_inp)
    sample = dist.sample()
    entropy = dist.entropy()

    print(f'fin input  (sample):', decode(sample.squeeze()))
    print(f'fin output (sample):', model.forward(F.one_hot(sample, N_VOCAB).to(torch.float32)))
    print(f'fin input entropy:', entropy)

-=-=- step 0 -=-=-
des class: IN (0)


attempt #0: 100%|██████████| 3000/3000 [00:16<00:00, 178.44it/s]


attempt #0 loss: -0.9995288848876953


attempt #1: 100%|██████████| 3000/3000 [00:15<00:00, 195.00it/s]


attempt #1 loss: -0.9999167919158936


...

attempt #2 loss: -0.9951199889183044


attempt #3: 100%|██████████| 3000/3000 [00:14<00:00, 208.87it/s]


attempt #3 loss: -0.9959519505500793


attempt #4: 100%|██████████| 3000/3000 [00:15<00:00, 199.25it/s]


attempt #4 loss: -0.9958794713020325
fin loss: -0.9961458444595337
fin output (from function): tensor([-0.8930,  2.9028,  0.8547, -4.5569,  1.3687, -5.6742, -4.1996,  9.4665])
fin input  (argmax):    q qqqqqqq
fin output (argmax): tensor([[-0.7942,  3.1887,  1.0007, -4.7220,  0.8489, -5.7898, -4.1859,  9.5906]],
       grad_fn=<AddmmBackward0>)
fin input  (sample):  qqqZqUqqqqq
fin output (sample): tensor([[-0.8638,  2.8850,  0.7848, -4.4925,  1.5249, -5.7696, -4.2774,  9.4701]],
       grad_fn=<AddmmBackward0>)
fin input entropy: tensor([[1.0411, 0.9175, 1.1635, 1.2277, 1.0803, 0.3053, 1.6267, 0.3843, 0.6132,
         0.8116, 0.4747, 1.1735]])
